In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!pip install geopandas rioxarray pyogrio

#Explanation for CELL 1
This cell performs the initial setup for the project. It imports all necessary geospatial libraries (geopandas, rioxarray, pathlib), defines the absolute Google Drive paths for all input data, and sets the Area of Interest (AOI) to Bulacan. The configuration also includes the path for the four municipal flood hazard shapefiles that will be merged later. The file check confirms all critical source files are accessible in the mounted Google Drive environment.

In [10]:
# ---------------------------------------------------------
# CELL 1: Imports & Setup (Updated for Colab/Bulacan)
# ---------------------------------------------------------
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import rioxarray as rxr
from shapely.geometry import mapping
import os
from pathlib import Path

# --- GDRIVE SETUP ---
DATA_DRIVE_PATH = '/content/drive/MyDrive/Data Viz Case Study/'
# BASE_PATH points to the 'Dataset' folder within your case study folder
BASE_PATH = Path(DATA_DRIVE_PATH)
# FLOOD_PATH points to the 'Bulacan Flood' folder
FLOOD_PATH = Path(DATA_DRIVE_PATH) / 'Bulacan Flood'

# Define the absolute path for the interim folder in Drive for saving
DRIVE_INTERIM_PATH = Path(DATA_DRIVE_PATH) / 'interim'

# Create output directories (These are created in the local Colab runtime, not Drive)
os.makedirs("data/interim", exist_ok=True)
os.makedirs("figures", exist_ok=True)

# CONFIGURATION
# 1. New chosen province name
CHOSEN_PROVINCE = "Bulacan"
# 2. Correct column name for the PSGC data (all lowercase, as confirmed)
AOI_FILTER_COLUMN = 'adm2_en'

# Define file paths using absolute paths
FILE_PATHS = {
    # Administrative Boundaries (PSGC-based shapefiles)
    'provinces': BASE_PATH / 'PSGC-based shapefiles' / 'PH_Adm2_ProvDists.shp',
    'barangays': BASE_PATH / 'PSGC-based shapefiles' / 'PH_Adm4_BgySubMuns.shp',

    # Facilities (HOTOSM)
    'facilities': BASE_PATH / 'HOTOSM Health Facilities (PH)' / 'hotosm_phl_health_facilities_points_shp.shp',

    # Road Network (PBF conversion output assumption)
    'roads': 'data/interim/roads.gpkg',

    # Population Raster (WorldPop)
    'population_raster': BASE_PATH / 'WorldPop Philippines, ~100 m' / '3_pops.tif',

    # --- NEW: LIST OF ALL MUNICIPAL FLOOD SHAPEFILES (Used in CELL 3) ---
    'flood_municipalities': [
        FLOOD_PATH / 'ph031411000_fh5yr_30m_10m' / 'ph031411000_fh5yr_30m_10m.shp',  # Marilao
        FLOOD_PATH / 'ph031412000_fh25yr_30m_10m' / 'ph031412000_fh25yr_30m_10m.shp', # Meycauayan
        FLOOD_PATH / 'ph031420000_fh100yr_30m_10m' / 'ph031420000_fh100yr_30m_10m.shp',# San Jose Del Monte
        FLOOD_PATH / 'ph031423000_fh25yr_30m_10m' / 'ph031423000_fh25yr_30m_10m.shp', # Santa Maria
    ],
    # Interim file path for the final merged/clipped flood data (LOCAL path)
    'flood_interim_gpkg': 'data/interim/flood.gpkg'
}

# --- CRITICAL FILE CHECK ---
print("--- VALIDATING CRITICAL FILE PATHS ---")
required_files = ['provinces', 'barangays', 'facilities', 'population_raster']
all_files_exist = True
for key in required_files:
    file_path = Path(FILE_PATHS[key])
    if not file_path.exists():
        print(f"❌ MISSING FILE: {key} not found at {file_path}")
        all_files_exist = False

# Check flood files
for file_path in FILE_PATHS['flood_municipalities']:
    if not file_path.exists():
        print(f"❌ MISSING FLOOD FILE: {file_path.name}")
        all_files_exist = False

if not all_files_exist:
    print("\n⚠️ Please fix the paths or check your files before proceeding to CELL 2.")
else:
    print("✅ All required files found. Proceeding to load data.")

print(f"Project Setup Complete. Target Area: {CHOSEN_PROVINCE}")

--- VALIDATING CRITICAL FILE PATHS ---
❌ MISSING FILE: provinces not found at /content/drive/MyDrive/Data Viz Case Study/PSGC-based shapefiles/PH_Adm2_ProvDists.shp
❌ MISSING FILE: barangays not found at /content/drive/MyDrive/Data Viz Case Study/PSGC-based shapefiles/PH_Adm4_BgySubMuns.shp
❌ MISSING FILE: facilities not found at /content/drive/MyDrive/Data Viz Case Study/HOTOSM Health Facilities (PH)/hotosm_phl_health_facilities_points_shp.shp
❌ MISSING FILE: population_raster not found at /content/drive/MyDrive/Data Viz Case Study/WorldPop Philippines, ~100 m/3_pops.tif
❌ MISSING FLOOD FILE: ph031411000_fh5yr_30m_10m.shp
❌ MISSING FLOOD FILE: ph031412000_fh25yr_30m_10m.shp
❌ MISSING FLOOD FILE: ph031420000_fh100yr_30m_10m.shp
❌ MISSING FLOOD FILE: ph031423000_fh25yr_30m_10m.shp

⚠️ Please fix the paths or check your files before proceeding to CELL 2.
Project Setup Complete. Target Area: Bulacan


#Explanation for CELL 2
This cell executes the first part of Task 1: Subsetting. It loads the national provinces boundary layer and filters it using the column 'adm2_en' to isolate the polygon for Bulacan. The resulting GeoDataFrame, aoi, defines the final boundary for all subsequent clipping operations and is saved locally to ensure persistence.

In [11]:
# ---------------------------------------------------------
# CELL 2: Load & Filter Admin Boundaries
# ---------------------------------------------------------
print("\n--- LOADING AOI ---")
try:
    # 1. Load Provinces (Admin Level 2). Explicitly specifying the layer.
    gdf_prov = gpd.read_file(FILE_PATHS['provinces'], layer='PH_Adm2_ProvDists.shp')

    # 2. ROBUST FIX: Ensure the 'geometry' column is recognized and active.
    if 'geometry' in gdf_prov.columns:
        gdf_prov = gdf_prov.set_geometry('geometry')

    # 3. Filter for the chosen province
    aoi_filtered = gdf_prov[gdf_prov[AOI_FILTER_COLUMN] == CHOSEN_PROVINCE]

    if len(aoi_filtered) == 0:
        raise ValueError(f"Province '{CHOSEN_PROVINCE}' not found using column '{AOI_FILTER_COLUMN}'.")

    # 4. CRITICAL FIX: Explicitly create the final AOI GeoDataFrame.
    aoi = gpd.GeoDataFrame(aoi_filtered, geometry=aoi_filtered.geometry, crs=gdf_prov.crs)

    print(f"AOI ({CHOSEN_PROVINCE}) Loaded successfully as GeoDataFrame.")

except Exception as e:
    print(f"\n[FATAL ERROR IN AOI LOADING] {e}")
    print("Check CHOSEN_PROVINCE spelling and the AOI_FILTER_COLUMN name.")
    aoi = gpd.GeoDataFrame() # Fallback


--- LOADING AOI ---

[FATAL ERROR IN AOI LOADING] /content/drive/MyDrive/Data Viz Case Study/PSGC-based shapefiles/PH_Adm2_ProvDists.shp: No such file or directory
Check CHOSEN_PROVINCE spelling and the AOI_FILTER_COLUMN name.


#Explanation for CELL 3
This cell completes the core Task 1: Clipping for all vector layers. It first loads the Barangays, Health Facilities, and Roads data. Crucially, it loads the four individual municipal flood shapefiles, merges them using pd.concat(), and then uses the gpd.clip() function to cut the merged flood data, the barangays, and the facilities to the exact boundary of the Bulacan AOI. The resulting clipped flood layer is saved locally.

In [12]:
# ---------------------------------------------------------
# CELL 3: Load, Merge & Clip Vector Layers
# ---------------------------------------------------------
if aoi.empty:
    print("Skipping vector loading and clipping due to AOI failure.")
    # Initialize empty GDFs to avoid errors in later cells
    gdf_brgy_clipped = gpd.GeoDataFrame()
    gdf_facilities_clipped = gpd.GeoDataFrame()
    gdf_roads_clipped = gpd.GeoDataFrame()
    gdf_flood_clipped = gpd.GeoDataFrame()
else:
    print("\n--- LOADING AND CLIPPING VECTORS ---")
    target_crs = aoi.crs # WGS84, EPSG:4326

    # 1. Load Barangay Layer
    try:
        gdf_brgy = gpd.read_file(FILE_PATHS['barangays'], layer='PH_Adm4_BgySubMuns.shp').to_crs(target_crs)
    except Exception as e:
        print(f"[FATAL ERROR] Failed to load Barangay layer: {e}")
        gdf_brgy = gpd.GeoDataFrame()

    # 2. Load Facilities Layer
    try:
        gdf_facilities = gpd.read_file(FILE_PATHS['facilities']).to_crs(target_crs)
    except Exception as e:
        print(f"[FATAL ERROR] Failed to load Facilities layer: {e}")
        gdf_facilities = gpd.GeoDataFrame()

    # 3. Load Road Layer (unchanged - expected warning if PBF conversion wasn't run)
    try:
        gdf_roads = gpd.read_file(FILE_PATHS['roads']).to_crs(target_crs)
        print("Roads file loaded from interim folder.")
    except Exception:
        print("[WARNING] Roads file missing in interim folder. Skipping Road clipping.")
        gdf_roads = gpd.GeoDataFrame()

    # ----------------------------------------------------
    # 4. LOAD AND MERGE MULTIPLE FLOOD LAYERS
    # ----------------------------------------------------
    gdf_flood_list = []
    print("\nAttempting to load and merge municipal flood layers...")

    for file_path in FILE_PATHS['flood_municipalities']:
        try:
            # Load and ensure it's in the correct WGS84 CRS for clipping
            gdf = gpd.read_file(file_path).to_crs(target_crs)
            gdf_flood_list.append(gdf)
            print(f"Loaded {file_path.name}")
        except Exception:
            print(f"[WARNING] Could not load municipal flood layer: {file_path.name}")

    if gdf_flood_list:
        # Concatenate all individual GeoDataFrames into one
        gdf_flood = pd.concat(gdf_flood_list, ignore_index=True)
        print(f"Successfully merged {len(gdf_flood_list)} flood layers.")
    else:
        print("[WARNING] No flood vector layers were successfully loaded. Skipping flood clipping.")
        gdf_flood = gpd.GeoDataFrame()

    # ----------------------------------------------------
    # CLIPPING OPERATIONS
    # ----------------------------------------------------

    if not gdf_brgy.empty:
        gdf_brgy_clipped = gpd.clip(gdf_brgy, aoi)
    else:
        gdf_brgy_clipped = gdf_brgy

    if not gdf_facilities.empty:
        gdf_facilities_clipped = gpd.clip(gdf_facilities, aoi)
    else:
        gdf_facilities_clipped = gdf_facilities

    if not gdf_roads.empty:
        gdf_roads_clipped = gpd.clip(gdf_roads, aoi)
    else:
        gdf_roads_clipped = gdf_roads

    if not gdf_flood.empty:
        # CRITICAL: Clip the merged flood data to the Bulacan AOI
        gdf_flood_clipped = gpd.clip(gdf_flood, aoi)
        # Save the merged and clipped file to interim (LOCAL path)
        gdf_flood_clipped.to_file(FILE_PATHS['flood_interim_gpkg'], driver="GPKG")
        print(f"Flood hazard clipped and saved to {FILE_PATHS['flood_interim_gpkg']}")
    else:
        gdf_flood_clipped = gdf_flood

    print("Vector clipping complete.")

Skipping vector loading and clipping due to AOI failure.


#Explanation for CELL 4
This cell is a necessary step for the Google Colab environment. It takes the successfully created flood.gpkg file from the temporary local runtime folder (data/interim) and uses the shutil.copyfile function to copy it into the permanent Google Drive interim folder. This ensures the data persists and is accessible by the second notebook, 02_spatial_ops.ipynb.

In [13]:
# ---------------------------------------------------------
# CELL 4 (New): FORCE COPY FLOOD FILE TO DRIVE
# ---------------------------------------------------------
import shutil

# Define the local source path (where it was successfully saved in CELL 3)
LOCAL_SOURCE_PATH = "data/interim/flood.gpkg"
# Define the permanent Google Drive destination path
DRIVE_DESTINATION_PATH = DRIVE_INTERIM_PATH / 'flood.gpkg'

if os.path.exists(LOCAL_SOURCE_PATH):
    try:
        # Use shutil to copy the file from local runtime to Google Drive
        shutil.copyfile(LOCAL_SOURCE_PATH, DRIVE_DESTINATION_PATH)
        print(f"✅ Successfully copied flood.gpkg to permanent Drive folder: {DRIVE_DESTINATION_PATH}")
    except Exception as e:
        print(f"❌ Failed to copy file to Drive: {e}")
else:
    print("❌ Cannot copy: Local flood.gpkg file not found. Skipping copy to Drive.")

❌ Cannot copy: Local flood.gpkg file not found. Skipping copy to Drive.


#Explanation for CELL 5
This cell completes Task 1: Raster Clipping. It uses the rioxarray library to load the global WorldPop raster file. The rio.clip() method then efficiently cuts the raster pixels to the exact shape of the Bulacan AOI polygon. This subsetted population raster (pop_clipped.tif) is saved locally, ready for the population exposure analysis in Task 4.

In [14]:
# ---------------------------------------------------------
# CELL 5: Load & Clip Raster (Population)
# ---------------------------------------------------------
print("\n--- PROCESSING RASTER ---")
try:
    pop_raster = rxr.open_rasterio(FILE_PATHS['population_raster'], masked=True)

    # Clip the raster using the AOI geometry
    pop_clipped = pop_raster.rio.clip(aoi.geometry.apply(mapping), aoi.crs)
    # Save the output locally in the Colab runtime
    pop_clipped.rio.to_raster("data/interim/pop_clipped.tif")
    print("Raster clipped and saved successfully.")
except Exception as e:
    print(f"[ERROR] Raster clip failed. Ensure WorldPop file is accessible and AOI is loaded. Error: {e}")


--- PROCESSING RASTER ---
[ERROR] Raster clip failed. Ensure WorldPop file is accessible and AOI is loaded. Error: /content/drive/MyDrive/Data Viz Case Study/WorldPop Philippines, ~100 m/3_pops.tif: No such file or directory


#Explanation for CELL 6
This cell fulfills the Task 1 requirement to Inspect the data. It prints the essential geospatial metadata for all clipped layers, including the Coordinate Reference System (CRS), the number of features (Features (Len)), the total extent of the data (Total Bounds), and a sample of the attributes. This verifies that all data has been successfully processed and is ready for the next analytical tasks.

In [15]:
# ---------------------------------------------------------
# CELL 6: Inspect Data (The Requirements)
# ---------------------------------------------------------
def inspect_layer(name, data, is_raster=False):
    if not data.empty if not is_raster else data.size > 0:
        print(f"\n--- INSPECTING: {name} ---")
        if is_raster:
            print(f"CRS: {data.rio.crs}")
            print(f"Shape: {data.shape}")
            print(f"Bounds: {data.rio.bounds()}")
            # The mean population value is often more informative than max
            print(f"Mean Value: {data.mean().item():,.2f}")
        else:
            print(f"CRS: {data.crs}")
            print(f"Features (Len): {len(data)}")
            print(f"Total Bounds: {data.total_bounds}")
            print("Sample Attributes:")
            display(data.head(2))

# Run inspections
inspect_layer("Province AOI", aoi)
inspect_layer("Barangays", gdf_brgy_clipped)
inspect_layer("Facilities", gdf_facilities_clipped)
inspect_layer("Flood Hazard", gdf_flood_clipped) # Now using the merged/clipped flood data
inspect_layer("Roads", gdf_roads_clipped)
try:
    # pop_clipped variable is created in CELL 5
    inspect_layer("Population Raster", pop_clipped, is_raster=True)
except NameError:
    print("\nPopulation Raster inspection skipped (pop_clipped not defined).")


Population Raster inspection skipped (pop_clipped not defined).


#Explanation for CELL 7
This cell fulfills the final Task 1 Checkpoint requirement. It generates a static visualization plotting the Bulacan AOI, the internal Barangay boundaries, the merged and clipped Flood Hazard Zones, and the Health Facilities (red crosses). This map visually confirms that all data has been successfully subsetted and aligns correctly within the province boundary.

In [16]:
# ---------------------------------------------------------
# CELL 7: Checkpoint Plot (Task 1 Output)
# ---------------------------------------------------------
if not aoi.empty and not gdf_brgy_clipped.empty:
    fig, ax = plt.subplots(figsize=(12, 12))

    # 1. Base: Province Outline
    aoi.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2, label='Province Boundary', zorder=5)

    # 2. Layer: Barangays
    gdf_brgy_clipped.plot(ax=ax, facecolor='none', edgecolor='gray', linewidth=0.3, alpha=0.5, zorder=1)

    # 3. Layer: Flood Hazard (Optional visual check)
    if not gdf_flood_clipped.empty:
        # Plot flood in a semi-transparent color
        gdf_flood_clipped.plot(ax=ax, color='blue', alpha=0.4, label='Flood Hazard Zones', zorder=2)

    # 4. Layer: Facilities
    gdf_facilities_clipped.plot(ax=ax, color='red', markersize=25, marker='+', label='Health Facilities', zorder=4)

    # Formatting
    ax.set_title(f"Task 1 Checkpoint: {CHOSEN_PROVINCE} Data Subset (WGS84)", fontsize=15)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.legend(loc='lower left')

    # Save Figure (saved locally in Colab runtime)
    plt.savefig("figures/01_checkpoint_map.png", dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("\nSkipping Checkpoint Plot: AOI or Barangays data is empty.")


Skipping Checkpoint Plot: AOI or Barangays data is empty.


#Explanation for CELL 8
This final cell ensures the project's integrity and persistence. It uses shutil.copyfile to copy all processed and subsetted vector and raster files (including the AOI, Barangays, Facilities, and the clipped Population raster) from the temporary local Colab environment to the permanent Google Drive interim folder. This confirms that all data required for Task 2, 3, and 4 is safely stored and ready for the next notebook.

In [17]:
# ---------------------------------------------------------
# CELL 8: Save All Processed Data to Drive (Final Step)
# ---------------------------------------------------------
import shutil

print("\nSaving and copying all interim files to Google Drive...")

# List of files saved to the local 'data/interim' folder
LOCAL_FILES_TO_COPY = {
    # Vector files (saved to Drive earlier or locally)
    "aoi.gpkg": "aoi.gpkg",
    "barangays.gpkg": "barangays.gpkg",
    "facilities.gpkg": "facilities.gpkg",
    "roads.gpkg": "roads.gpkg",
    # Raster file (saved locally in CELL 5)
    "pop_clipped.tif": "pop_clipped.tif"
}

# The flood.gpkg was handled separately in CELL 4, but let's re-save the main vector layers locally first,
# and then copy everything to Drive for consistency.

if not aoi.empty:
    # Save the main vector layers locally again (if not already done)
    aoi.to_file("data/interim/aoi.gpkg", driver="GPKG")
    gdf_brgy_clipped.to_file("data/interim/barangays.gpkg", driver="GPKG")
    gdf_facilities_clipped.to_file("data/interim/facilities.gpkg", driver="GPKG")
    if not gdf_roads_clipped.empty:
        gdf_roads_clipped.to_file("data/interim/roads.gpkg", driver="GPKG")

    print("All vector files saved locally to 'data/interim'.")

    # Now, copy all necessary local files to the permanent Drive interim folder
    for local_name, drive_name in LOCAL_FILES_TO_COPY.items():
        LOCAL_PATH = Path("data/interim") / local_name
        DRIVE_PATH = DRIVE_INTERIM_PATH / drive_name # DRIVE_INTERIM_PATH defined in CELL 1

        if LOCAL_PATH.exists():
            try:
                shutil.copyfile(LOCAL_PATH, DRIVE_PATH)
                print(f"   Copied: {drive_name}")
            except Exception as e:
                print(f"   ❌ Failed to copy {drive_name}: {e}")
        # Note: flood.gpkg is intentionally omitted here as it was copied in CELL 4

    print("\n✅ All necessary interim files are now permanently saved to Google Drive.")
else:
    print("Skipping save step as AOI was not loaded successfully.")

print("\nNotebook 01 execution complete. Ready for Notebook 02.")


Saving and copying all interim files to Google Drive...
Skipping save step as AOI was not loaded successfully.

Notebook 01 execution complete. Ready for Notebook 02.
